In [133]:
import numpy as np
import pandas as pd 
import networkx as nx
import numpy.random as rd
from scipy.optimize import minimize 

In [2]:
%run rome.py
romes = list(graph.keys())

In [35]:
g = nx.DiGraph(graph)
distance = {}
for rome in graph.keys():
    distance[rome] = {}
    distance[rome][rome] = 0
    for connex in (x for x in romes if x != rome):
        try: 
            distance[rome][connex] = nx.shortest_path_length(g,rome,connex)
        except:
            distance[rome][connex] = 20

In [96]:
class JobSeeker: 
    def __init__(self,rome,rho,T):
        self.rome = rome 
        self.rho = rho
        self.T = T
        
class Branch: 
    def __init__(self,rome,rho,m,hirings):
        self.rho = rho
        self.m = m 
        self.hirings = hirings
        self.rome = rome 
        
class Firm: 
    def __init__(self,rho,m,branches):
        self.rho = rho
        self.m = m 
        self.branches = branches
        self.romes = list(branches.keys())
        self.nb_branches = len(branches)
        self.tot_hirings = sum(branches.values())
    
    def split(self):
        return [Branch(rome,self.rho,self.m,h) for rome, h in self.branches.items()]
        
def random_firm(rho,m,max_branches,hirings_per_branch): 
    branches = rd.choice(romes,rd.randint(1,high=max_branches))
    hirings = {b:rd.chisquare(hirings_per_branches) for b in branches} 
    return Firm(rd.choice(rho),rd.choice(m),hirings)


In [245]:
rd.seed(123)
N = 50
F = 5
DE = {}
rho = [0.9,0.5,0.1]
m = [1.0,0.5,0]
T = [6,3,0]
max_branches = 3
hirings_per_branches = 2

DE = {n:JobSeeker(rd.choice(romes),rd.choice(rho),rd.choice(T)) for n in range(1,N+1)}
FIRMS = {f:random_firm(rho,m,max_branches,hirings_per_branches) for f in range(1,F+1)}

In [246]:
def alphas(beta,DE,FIRMS):
    alpha = {}
    for n,de in DE.items():
        tot = 0
        alpha[n] = {}
        X = [de.rho]
        for f,firm in FIRMS.items():
            X = X[0:1] + [firm.rho, firm.m]
            for b,rome in enumerate(firm.romes):
                d = distance[de.rome][rome]
                X = X[0:3] + [d,de.rho**d,firm.rho**d] # X = [de.rho,firm.rho,firm.m,distance,de.rho*distance,firm.rho*distance]
                a = np.exp(np.dot(beta,X))
                tot += a 
                alpha[n][(f,b)] = a
        for f,firm in FIRMS.items():
            for b in range(firm.nb_branches):
                alpha[n][(f,b)] = alpha[n][(f,b)]/tot
    return alpha

def pass_through(alpha,DE,FIRMS):
    pi = {}
    for f,firm in FIRMS.items():
        pi[f] = {}
        for b,rome in enumerate(firm.romes):
            pi[f][b] = np.sum([(1.0 - (1.0-alpha[n][(f,b)])**de.T)*firm.rho**distance[de.rome][rome] for n,de in DE.items()])
            pi[f][b] = firm.m*firm.branches[rome]/(pi[f][b]+firm.branches[rome])
    return pi

def matches(beta,DE,FIRMS):
    m = 0.0
    alpha = alphas(beta,DE,FIRMS)
    pi = pass_through(alpha,DE,FIRMS)
    for n,de in DE.items():
        for f,firm in FIRMS.items():
            for b,rome in enumerate(firm.romes):
                d = distance[de.rome][rome]
                m += (firm.rho**d)*pi[f][b]*(de.rho**d)*(1.0 - (1.0-alpha[n][(f,b)])**de.T)
    return m

def recommendations(beta,DE,FIRMS):
    R = {}
    alpha = alphas(beta,DE,FIRMS)
    for n,de in DE.items():
        firm_branches = list(alpha[n].keys())
        index = list(range(len(firm_branches)))
        R[n] = {}
        for r in range(1,de.T+1):
            R[n][r] = firm_branches[rd.choice(index,size=1,p=list(alpha[n].values()))[0]]
            alpha[n] = {fb:v for fb, v in alpha[n].items() if fb != R[n][r]}
            tot = sum(alpha[n].values())
            alpha[n] = {fb:v/tot for fb, v in alpha[n].items()}
            firm_branches = list(alpha[n].keys())
            index = list(range(len(firm_branches)))
    return {n:{r:(fb[0],FIRMS[fb[0]].romes[fb[1]]) for r, fb in R[n].items()} for n in DE.keys()}


In [247]:
res = minimize(lambda x: -matches(x,DE,FIRMS),np.zeros(6),method='BFGS')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


In [248]:
res.x

array([-3.53785884e-04, -7.92078225e+02,  3.27015558e+02, -1.00819360e+04,
        5.17607405e+02,  1.31172277e+03])

In [249]:
alphas(np.ones(6),DE,FIRMS)

{1: {(1, 0): 2.577146693954619e-07,
  (2, 0): 0.49999814566012146,
  (2, 1): 0.49999814566012146,
  (3, 0): 1.7169832978740544e-07,
  (4, 0): 1.7169832978740544e-07,
  (5, 0): 3.0841127397406723e-06,
  (5, 1): 2.3455688380829032e-08},
 2: {(1, 0): 9.643177397327512e-07,
  (2, 0): 0.4999988988576598,
  (2, 1): 0.4999988988576598,
  (3, 0): 6.56442986823925e-07,
  (4, 0): 2.5467312136071585e-07,
  (5, 0): 9.278443002703765e-08,
  (5, 1): 2.3406640235130506e-07},
 3: {(1, 0): 1.6498031893373566e-06,
  (2, 0): 0.49999828894226256,
  (2, 1): 0.49999828894226256,
  (3, 0): 4.5948876021380715e-07,
  (4, 0): 2.2881169520021772e-08,
  (5, 0): 1.1345830897053678e-06,
  (5, 1): 1.553592660885201e-07},
 4: {(1, 0): 1.9887970357488534e-08,
  (2, 0): 0.4999991476293234,
  (2, 1): 0.4999991476293234,
  (3, 0): 4.0153119651578565e-09,
  (4, 0): 1.2539091924981488e-06,
  (5, 0): 4.2231017010103045e-07,
  (5, 1): 4.618708147759007e-09},
 5: {(1, 0): 1.1131109336269089e-05,
  (2, 0): 0.49998045908908384,

In [209]:
D = {n:{r:distance[de.rome][fb[1]] for r,fb in R[n].items()} for n,de in DE.items()}

In [211]:
graph[DE[13].rome]

[]

In [216]:
firm_romes = []
for firm in FIRMS.values():
    firm_romes += firm.romes
sum([DE[n].rome in firm_romes for n in DE.keys()])/len(DE)

0.05

NameError: name 'unique' is not defined